### Main Libraries Needed

In [1]:
#Import of main libraries

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np 
from __future__ import division, print_function
from pandas import read_csv
from pandas import read_excel
from pandas import DataFrame
from pandas import ExcelWriter
from pandas import ExcelFile

#Import of the pyomo module
from pyomo.environ import *

### Importing and creating necessary Data Frames

In [2]:
#Import Bids, Lanes, and Carrier DF's
bidsDf = read_csv("UK Bids.csv")
bidsDf = bidsDf.drop("Unnamed: 0", axis = 1)
lanesDf = read_csv("UK Lanes.csv")
lanesDf = lanesDf.drop("Unnamed: 0", axis = 1)
CarriersDf = read_csv("Carriers.csv")
CarriersDf = CarriersDf.drop("Unnamed: 0", axis = 1)

In [3]:
#Create data frames for delta matrix
deltaDf = DataFrame(np.zeros((len(lanesDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    deltaDf.at[bidsDf.loc[bid,'Cluster Lot Index'] - 1, bid] = 1

In [4]:
# Create the eta DF
etaDf = DataFrame(np.zeros((len(CarriersDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    etaDf.at[bidsDf.loc[bid,'Carrier Index'] - 1, bid] = 1

In [5]:
import pandas as pd

In [6]:
Mc = pd.Series(np.sum(etaDf, axis = 1))

In [7]:
Mc

0     9.0
1     9.0
2     9.0
3     9.0
4     6.0
5     9.0
6     0.0
7     9.0
8     7.0
9     6.0
10    2.0
11    0.0
12    9.0
13    5.0
14    0.0
15    0.0
dtype: float64

### Create the necessary dictionaries

In [8]:
carriers = Mc.to_dict()

bidValues = dict()
for bid in bidsDf.index:
    bidValues[bid] = bidsDf.loc[bid, 'Cost']

demandValues = dict()
for lane in lanesDf.index:
    demandValues[lane] = 1

delta = dict()
for lane in lanesDf.index:
    for bid in bidsDf.index:
        delta[(lane,bid)] = deltaDf.loc[lane,bid]

eta = dict()
for c in CarriersDf.index:
    for bid in bidsDf.index:
        eta[(c,bid)] = etaDf.loc[c,bid]


In [9]:
# Create the Concrete Model
model = ConcreteModel()

model.numBids = len(bidsDf.index)

model.numItems = len(lanesDf.index)

model.numCarriers = len(CarriersDf.index)

model.BIDS = Set(initialize = bidsDf.index.values)

model.LANES = Set(initialize = lanesDf.index.values)

model.CARRIERS = Set( initialize = CarriersDf.index.values)

model.M = Param(model.CARRIERS, initialize = carriers)

model.bidValue = Param(model.BIDS, initialize = bidValues, doc='Value of each bid in the program')

model.demand = Param(model.LANES, initialize = demandValues, doc='Total demand on each lane')

# Below is where I made the suggested work around from https://stackoverflow.com/questions/45616967/pyomo-valueerror-invalid-constraint-expression

model.delta = Param(model.LANES, model.BIDS, initialize= delta, doc='delta gives information regarding which lanes are in a bid package')#,mutable = True)

model.eta = Param(model.CARRIERS, model.BIDS, initialize= eta, doc='eta gives information regarding which carriers are in a bid package', mutable = True)

model.x = Var(model.BIDS, domain = Binary, doc='Decision variable for each bid in the program')

model.z = Var(model.CARRIERS, domain = Binary, doc='Decision variable for each carrier in the program')

model.maxCarriers = 5
model.minCarriers = 1

### Set Model Obj and Constraints

In [10]:
def obj_expression(model):
    return sum(model.bidValue[i]*model.x[i] for i in model.BIDS)
model.OBJ = Objective(rule=obj_expression, sense=minimize, doc='Objective function definition')

In [11]:
def constraint_rule(model, l):
    return sum(model.delta[l,b]*model.x[b] for b in model.BIDS) <= model.demand[l]
model.xConstraint = Constraint(model.LANES, rule=constraint_rule)

def demand_constraint_rule(model):
    return sum(model.x[b] for b in model.BIDS) >= model.numItems
model.demandConstraint = Constraint(rule=demand_constraint_rule)

def constraint2_rule(model, k):
    return sum(model.eta[k,b]*model.x[b] for b in model.BIDS) - model.M[k]*model.z[k]<=0
model.upperBoundConstraint = Constraint(model.CARRIERS, rule=constraint2_rule)

def constraint3_rule(model, k):
    return model.z[k] - sum(model.eta[k,b]*model.x[b] for b in model.BIDS)<=0
model.lowerBoundConstraint = Constraint(model.CARRIERS, rule=constraint3_rule)

def constraint4_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)<=model.maxCarriers
model.zConstraint = Constraint(rule=constraint4_rule)

def constraint5_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)>=model.minCarriers
model.zConstraint2 = Constraint(rule=constraint5_rule)

### Run the Model

In [12]:
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [13]:
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
%timeit results = opt.solve(model)

89.7 ms ± 7.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
results = opt.solve(model)
model.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 25192.8
  Upper bound: 25192.8
  Number of objectives: 1
  Number of constraints: 45
  Number of variables: 106
  Number of nonzeros: 417
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.03034663200378418
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    OBJ:
      Value: 25192.8
  Variable:
    x[0]:
      Value: 1
    x[11]:
      Value: 1
    x[21]:
      Value: 1
    x[30]:
      Value: 1
    x[39]:
      Value: 1
    x[48]:
      Value: 1
    x[59]:
      Value: 1
    x[71]:
      Value: 1
    x[79]:
      Value: 1
    z[13]:
      Value: 1
    z[2]:
      Value: 1
    z[4]:
      Value: 1
  Constraint: No values



In [15]:
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 25192.8
  Upper bound: 25192.8
  Number of objectives: 1
  Number of constraints: 45
  Number of variables: 106
  Number of nonzeros: 417
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.03034663200378418
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [0]:
#model.pprint()

In [16]:
winningBids = []
index = 0
bidNum = 0
for bids in range(len(bidsDf.index)):
    if model.x[bids].value > 0:
        winningBids.append(bidNum)
    bidNum += 1
    index += 1

winningBidsDf = bidsDf.iloc[winningBids]
winningBidsDf.head()

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,ITLW@ISO_057,ITLW,ISO_057,57,3900.0,1,3900.0,0.0,41,UK,5,1
11,SUTI@ISO_058,SUTI,ISO_058,58,3232.8,1,3232.8,0.0,10,UK,14,2
21,ITLW@ISO_059,ITLW,ISO_059,59,2220.0,1,2220.0,0.0,9,UK,5,3
30,DNHG@ISO_060,DNHG,ISO_060,60,2802.0,1,2802.0,0.0,2,UK,3,4
39,DNHG@ISO_061,DNHG,ISO_061,61,2844.0,1,2844.0,0.0,7,UK,3,5


In [17]:
winningBidsDf

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,ITLW@ISO_057,ITLW,ISO_057,57,3900.0,1,3900.0,0.0,41,UK,5,1
11,SUTI@ISO_058,SUTI,ISO_058,58,3232.8,1,3232.8,0.0,10,UK,14,2
21,ITLW@ISO_059,ITLW,ISO_059,59,2220.0,1,2220.0,0.0,9,UK,5,3
30,DNHG@ISO_060,DNHG,ISO_060,60,2802.0,1,2802.0,0.0,2,UK,3,4
39,DNHG@ISO_061,DNHG,ISO_061,61,2844.0,1,2844.0,0.0,7,UK,3,5
48,ITLW@ISO_062,ITLW,ISO_062,62,1380.0,1,1380.0,0.0,10,UK,5,6
59,DNHG@ISO_063,DNHG,ISO_063,63,1306.8,1,1306.8,0.0,345,UK,3,7
71,DNHG@ISO_064,DNHG,ISO_064,64,4226.4,1,4226.4,0.0,130,UK,3,8
79,SUTI@ISO_217,SUTI,ISO_217,217,3280.8,1,3280.8,0.0,55,UK,14,9


In [0]:
Total = sum(winningBidsDf['Cost']*winningBidsDf['Shipments per Lot'])
Total

In [18]:
winningBidsDf.to_csv('UK Carrier Constraint WinningBids.csv')